In [4]:
import pickle
import re
import string
import nltk
import pandas as pd
import spacy
import re
import string
from collections import Counter
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
spacy.require_gpu()
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [7]:
data_df = pd.read_pickle("askreddit_lemmatized.pickle")
data_df

,0
0,victorias secret originally suppose place man ...
1,ford mustang suppose car big trunk fit groce...
2,heel remember correctly
3,family feud get new question answer
4,skirt wear battle man room
...,...
101136,maybe
101137,cake day wonder people know
101138,s cake emoji username comment cake day check p...
101139,think problem be narwhal edit thank sorry get ...


In [12]:
ar = data_df[0].tolist()

In [1]:
def lemmatize(text):
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    nlp.max_length = 14082871 
    result = nlp(text) 
    lemmed = [word.lemma_ for word in result if not word.is_stop]
    return lemmed

In [2]:
def pre_lemma(documents):
    lemmed_list = []
    for doc in documents:
#         lemmed_list.append(doc)
        lemmed_list.append(lemmatize(doc))
    return lemmed_list

In [13]:
# # Apply a first round of text cleaning techniques
# import re
# import string



# round1 = lambda x: pre_lemma(x)

# data_clean = pd.DataFrame(data_df[2:3].apply(round1))
# data_clean

In [9]:
# data_clean.to_pickle("ama_lemmatized.pickle")

In [14]:
# for value in data_df[2:3]:
#     data[key] = pre_lemma(value)

In [ ]:
data_df = pd.DataFrame.from_dict(data, orient='index').fillna(value='none')
# data_df.columns = ['text']
# data_df = data_df.sort_index()
data_df
data_df.to_pickle("individual_comments_lemmatized.pickle")

In [ ]:
pd.set_option('max_colwidth',150)

data_df = pd.DataFrame.from_dict(data, orient='index').fillna(value='none')
# data_df.columns = ['text']
# data_df = data_df.sort_index()
data_df

In [ ]:
pd.set_option('max_colwidth',150)

data_df_nolemma = pd.DataFrame.from_dict(data2, orient='index').fillna(value='none')
# data_df.columns = ['text']
# data_df = data_df.sort_index()
data_df_nolemma

In [15]:
import multiprocessing
from gensim.models import Word2Vec
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

16

In [25]:
stop_list = ['of', 'it', 'to', 'for', 'you', 'the', 'that', 'if', 'my', 'and', 'in', 'from','going', 'too', 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa', 'aaaaaaaaaaaaaaaaaaaaaaaa', 'aaaaaaaaaaaa', 'aaaaaaaa', 'aaa', 'aaaaaaaaahhhhhhhhhhhh', 'aaaaaaaand', 'aaaaaaaauuuuuugggghhh', 'aaaaaaafternoon', 'aaaaaaand', 'aaaaaand', 'aaaaall', 'aaaaalllllll', 'aaaaand', 'is', 'are', 'what', 'they', 'we', 'about', 'me', 'have', 'im', 'this', 'yeah','will','can','get','be', 'have','be', 'but', 'he', 'she', 'her', 'was', 'with', 'they']

In [27]:
vectorizer = TfidfVectorizer(stop_words=stop_list)

doc_word = vectorizer.fit_transform(ar)
data_dtmn = pd.DataFrame(doc_word.toarray(), columns=vectorizer.get_feature_names()).T

In [28]:
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [29]:
from time import time
t = time()

w2v_model.build_vocab(ar, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 13:05:17: collecting all words and their counts
WARNING - 13:05:17: Each 'sentences' item should be a list of words (usually unicode strings). First item here is instead plain <class 'str'>.
INFO - 13:05:17: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 13:05:17: PROGRESS: at sentence #10000, processed 642369 words, keeping 132 word types
INFO - 13:05:18: PROGRESS: at sentence #20000, processed 1470223 words, keeping 195 word types
INFO - 13:05:18: PROGRESS: at sentence #30000, processed 2238263 words, keeping 233 word types
INFO - 13:05:18: PROGRESS: at sentence #40000, processed 2804435 words, keeping 287 word types
INFO - 13:05:18: PROGRESS: at sentence #50000, processed 4079390 words, keeping 342 word types
INFO - 13:05:18: PROGRESS: at sentence #60000, processed 4882560 words, keeping 376 word types
INFO - 13:05:18: PROGRESS: at sentence #70000, processed 5329028 words, keeping 511 word types
INFO - 13:05:18: PROGRESS: at sentence #80000, processe

Time to build vocab: 0.01 mins


In [30]:
t = time()

w2v_model.train(ar, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 13:05:21: training model with 15 workers on 85 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=20 window=2
INFO - 13:05:21: worker thread finished; awaiting finish of 14 more threads
INFO - 13:05:21: worker thread finished; awaiting finish of 13 more threads
INFO - 13:05:21: worker thread finished; awaiting finish of 12 more threads
INFO - 13:05:21: worker thread finished; awaiting finish of 11 more threads
INFO - 13:05:21: worker thread finished; awaiting finish of 10 more threads
INFO - 13:05:21: worker thread finished; awaiting finish of 9 more threads
INFO - 13:05:21: worker thread finished; awaiting finish of 8 more threads
INFO - 13:05:21: worker thread finished; awaiting finish of 7 more threads
INFO - 13:05:21: worker thread finished; awaiting finish of 6 more threads
INFO - 13:05:21: worker thread finished; awaiting finish of 5 more threads
INFO - 13:05:21: worker thread finished; awaiting finish of 4 more threads
INFO - 13:05:21: worker thread finish

INFO - 13:05:25: worker thread finished; awaiting finish of 6 more threads
INFO - 13:05:25: worker thread finished; awaiting finish of 5 more threads
INFO - 13:05:25: worker thread finished; awaiting finish of 4 more threads
INFO - 13:05:25: worker thread finished; awaiting finish of 3 more threads
INFO - 13:05:25: worker thread finished; awaiting finish of 2 more threads
INFO - 13:05:25: worker thread finished; awaiting finish of 1 more threads
INFO - 13:05:25: worker thread finished; awaiting finish of 0 more threads
INFO - 13:05:25: EPOCH - 7 : training on 7200477 raw words (276115 effective words) took 0.7s, 406308 effective words/s
INFO - 13:05:26: worker thread finished; awaiting finish of 14 more threads
INFO - 13:05:26: worker thread finished; awaiting finish of 13 more threads
INFO - 13:05:26: worker thread finished; awaiting finish of 12 more threads
INFO - 13:05:26: worker thread finished; awaiting finish of 11 more threads
INFO - 13:05:26: worker thread finished; awaiting f

INFO - 13:05:30: worker thread finished; awaiting finish of 13 more threads
INFO - 13:05:30: worker thread finished; awaiting finish of 12 more threads
INFO - 13:05:30: worker thread finished; awaiting finish of 11 more threads
INFO - 13:05:30: worker thread finished; awaiting finish of 10 more threads
INFO - 13:05:30: worker thread finished; awaiting finish of 9 more threads
INFO - 13:05:30: worker thread finished; awaiting finish of 8 more threads
INFO - 13:05:30: worker thread finished; awaiting finish of 7 more threads
INFO - 13:05:30: worker thread finished; awaiting finish of 6 more threads
INFO - 13:05:30: worker thread finished; awaiting finish of 5 more threads
INFO - 13:05:30: worker thread finished; awaiting finish of 4 more threads
INFO - 13:05:30: worker thread finished; awaiting finish of 3 more threads
INFO - 13:05:30: worker thread finished; awaiting finish of 2 more threads
INFO - 13:05:30: worker thread finished; awaiting finish of 1 more threads
INFO - 13:05:30: work

INFO - 13:05:34: worker thread finished; awaiting finish of 3 more threads
INFO - 13:05:34: worker thread finished; awaiting finish of 2 more threads
INFO - 13:05:34: worker thread finished; awaiting finish of 1 more threads
INFO - 13:05:34: worker thread finished; awaiting finish of 0 more threads
INFO - 13:05:34: EPOCH - 20 : training on 7200477 raw words (275510 effective words) took 0.7s, 409772 effective words/s
INFO - 13:05:35: worker thread finished; awaiting finish of 14 more threads
INFO - 13:05:35: worker thread finished; awaiting finish of 13 more threads
INFO - 13:05:35: worker thread finished; awaiting finish of 12 more threads
INFO - 13:05:35: worker thread finished; awaiting finish of 11 more threads
INFO - 13:05:35: worker thread finished; awaiting finish of 10 more threads
INFO - 13:05:35: worker thread finished; awaiting finish of 9 more threads
INFO - 13:05:35: worker thread finished; awaiting finish of 8 more threads
INFO - 13:05:35: worker thread finished; awaiting

INFO - 13:05:39: worker thread finished; awaiting finish of 10 more threads
INFO - 13:05:39: worker thread finished; awaiting finish of 9 more threads
INFO - 13:05:39: worker thread finished; awaiting finish of 8 more threads
INFO - 13:05:39: worker thread finished; awaiting finish of 7 more threads
INFO - 13:05:39: worker thread finished; awaiting finish of 6 more threads
INFO - 13:05:39: worker thread finished; awaiting finish of 5 more threads
INFO - 13:05:39: worker thread finished; awaiting finish of 4 more threads
INFO - 13:05:39: worker thread finished; awaiting finish of 3 more threads
INFO - 13:05:39: worker thread finished; awaiting finish of 2 more threads
INFO - 13:05:39: worker thread finished; awaiting finish of 1 more threads
INFO - 13:05:39: worker thread finished; awaiting finish of 0 more threads
INFO - 13:05:39: EPOCH - 27 : training on 7200477 raw words (276222 effective words) took 0.7s, 416524 effective words/s
INFO - 13:05:40: worker thread finished; awaiting fin

Time to train the model: 0.34 mins


In [31]:
w2v_model.init_sims(replace=True)

INFO - 13:06:06: precomputing L2-norms of word weight vectors


In [33]:
w2v_model.wv.similarity('red', 'green')

KeyError: "word 'red' not in vocabulary"

In [ ]:
w2v_model.wv.similarity('science', 'career')

In [23]:
w2v_model.wv.doesnt_match(['bad', 'three', 'two'])

WARNING - 12:50:51: vectors for words {'bad', 'two', 'three'} are not present in the model, ignoring these words


ValueError: cannot select a word from an empty list

In [ ]:
w2v_model.wv.most_similar(positive=["europe", "america"], negative=["russia"], topn=3)

In [ ]:
w2v_model.wv.most_similar(positive=["europe", "america"], negative=["china"], topn=3)

In [ ]:
w2v_model.wv.most_similar(positive=["europe", "america"], negative=["britain"], topn=3)

In [ ]:
w2v_model.wv.most_similar(positive=["app", "twitter"], negative=["app"], topn=3)